<a href="https://colab.research.google.com/github/aleks-haksly/Postypashka/blob/main/DA/01%20HW/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Всё необходимое для выполнения данной лабораторной работы!
import requests
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', 100)

## Задание 1. Парсинг - 10 баллов

В этом задании Вам предстоит работать с API сайта <a target="_blank" href="hh.ru">hh.ru</a>. Документация доступна по <a target="_blank" href="https://github.com/hhru/api/blob/master/docs/vacancies.md#search">ссылке</a>.

**Задача:**
1. выбрать 5 интересующих вас профессий (не смежных, т.е. Аналатик и Data Engineer не считается корректным)
2. затем получить доступные по данным профессиям вакансии в Москве и выделить их физические координаты
3. нанести полученные координаты на карту местоположения компаний

In [3]:
URL_areas = 'https://api.hh.ru/areas'
URL_professions = 'https://api.hh.ru/professional_roles'
URL_vacancies = 'https://api.hh.ru/vacancies'

In [4]:
data_areas = requests.get(URL_areas).json()
sleep(0.5)
data_professions = requests.get(URL_professions).json()

In [5]:
def get_city_id_by_name(data:dict=data_areas, county_name:str='Россия', city_name:str='Москва') -> str | None:
    """
    Returns the city ID given a country name and city name.

    :param data: A dictionary containing country and city data.
    :param county_name: The name of the country.
    :param city_name: The name of the city.
    :return: The ID of the city if found, otherwise None
    """
    for country in data_areas:
        if country.get('name', []) == county_name:
            for city in country['areas']:
                if city.get('name', []) == city_name:
                    return city.get('id', None)

In [6]:
def get_random_professions_names(data:dict=data_professions, n:int=5) -> list:
    """
    Returns a list of random profession names.

    :param data: A dictionary containing professions data.
    :param n: The number of profession names to return.
    :return: A list of random profession names
    """
    result = []
    profession_categories = [profession_categories.get('name') for profession_categories in data.get('categories')][:-1] #Категорию "другое" не рассматриваем
    # Выберем по 1 случайной профессии из n случайных категорий
    for profession_category in np.random.choice(profession_categories, size=n, replace=False):
        for profession in data_professions.get('categories'):
            if profession.get('name', []) == profession_category:
                result.append(np.random.choice(profession.get('roles'), size=1, replace=False)[0].get('name'))
                break
    return result


*Пример запроса для профессии:*

In [45]:
def get_vacancies_page(vacancy_name:str, page:int=0, city:str='Москва') -> dict:
    """
    Fetches a page of vacancies based on the vacancy name and city.

    :param vacancy_name: The name of the vacancy to search for.
    :param page: The page number to fetch (defaults to 0).
    :param city: The city to filter vacancies by (defaults to 'Москва').
    :return: A JSON object containing the vacancies or None if an error occurs.
    """
    params = {
        'text': vacancy_name,
        'area': get_city_id_by_name(city_name=city),
        'page': page,
        'per_page': 10
    }
    try:
        response = requests.get(URL_vacancies, params)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching vacancies: {e}")
        return None
    return response.json()

In [59]:
def get_vacancies(vacancy_name, city='Москва'):
    """
    Retrieves all vacancies matching the specified vacancy name and city by paginating through the API results.

    :param vacancy_name: The name of the vacancy to search for.
    :param city: The city to filter the vacancies by (defaults to 'Москва').
    :return: A dictionary containing a list of vacancies and the total number found.
    """

    page = 0
    vacancies = []
    current_page = get_vacancies_page(vacancy_name=vacancy_name, page=page, city=city)

    if not current_page:
        return {'items': [], 'found': 0}

    total_pages = current_page.get('pages', 0)

    while total_pages > page:
        vacancies.extend(current_page.get('items', []))
        page += 1
        sleep(np.random.rand() + 0.5)
        current_page = get_vacancies_page(vacancy_name=vacancy_name, page=page, city=city)
        if not current_page:
          break
    return {'items': vacancies, 'found': len(vacancies)}

In [63]:
np.random.seed(42)
data = []
for vacancy_name in get_random_professions_names(n=5):
    vacancies = get_vacancies(vacancy_name=vacancy_name)
    data.extend(vacancies.get('items'))
    print(f'{vacancies.get("found")} vacancies for {vacancy_name} fetched')

8 vacancies for Артист, актер, аниматор fetched
52 vacancies for Агроном fetched
115 vacancies for Автомойщик fetched
555 vacancies for Финансовый директор (CFO) fetched
Error fetching vacancies: 400 Client Error: Bad Request for url: https://api.hh.ru/vacancies?text=%D0%9F%D1%81%D0%B8%D1%85%D0%BE%D0%BB%D0%BE%D0%B3&area=1&page=200&per_page=10
2000 vacancies for Психолог fetched


In [65]:
len(data)

2730

In [213]:
df1 = df.copy()

In [211]:
df1['area'] = df1.area.apply(lambda x: x.get('name'))
df1['salary_from'] = df1.salary.apply(lambda x: x.get('from', None) if x else None)
df1['salary_to'] = df1.salary.apply(lambda x: x.get('to', None) if x else None)
df1['salary_currency'] = df1.salary.apply(lambda x: x.get('currency', None) if x else None)
df1['salary_gross'] = df1.salary.apply(lambda x: x.get('gross', None) if x else None)
df1.drop(columns=['salary'], inplace=True)
df1.type	= df1.type.apply(lambda x: x.get('name', None) if x else None)
df1['city_address'] = df1.address.apply(lambda x: x.get('city', None) if x else None)
df1['street_adress'] = df1.address.apply(lambda x: x.get('street', None) if x else None)
df1['metro_station'] = df1.address.apply(lambda x: x.get('metro_station', None)
df1['metro_line'] = df1.address.apply(lambda x: x.get('metro_line', None)
df1['metro_distance'] = df1.address.apply(lambda x: x.get('metro_distance', None)
df1['r']
#df1.drop(columns=['address'], inplace=True)


In [212]:
df1

,id,premium,name,department,has_test,response_letter_required,area,type,response_url,sort_point_distance,published_at,created_at,archived,apply_alternate_url,show_logo_in_search,insider_interview,url,alternate_url,relations,employer,snippet,contacts,schedule,working_days,working_time_intervals,working_time_modes,accept_temporary,professional_roles,accept_incomplete_resumes,experience,employment,adv_response_url,is_adv_vacancy,adv_context,salary_from,salary_to,salary_currency,salary_gross,city_address,street_adress
0,107214681,False,Главный агроном,None,False,False,Москва,Открытая,None,None,2024-09-12T17:36:02+0300,2024-09-12T17:36:02+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,NaN,None,https://api.hh.ru/vacancies/107214681?host=hh.ru,https://hh.ru/vacancy/107214681,[],"{'id': '1069209', 'name': 'Кадровый центр През...",{'requirement': 'Высшее профильное образование...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False,"[{'id': '7', 'name': 'Агроном'}]",False,"{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",None,False,None,180000,NaN,RUR,False,None,None
1,106335339,False,Управляющий агрофермой (Агроном) релокация на ...,None,False,True,Москва,Открытая,None,None,2024-08-26T14:39:27+0300,2024-08-26T14:39:27+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,NaN,None,https://api.hh.ru/vacancies/106335339?host=hh.ru,https://hh.ru/vacancy/106335339,[],"{'id': '11217206', 'name': 'Всё, я увольняюсь!...",{'requirement': 'Профильное образование. Релев...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False,"[{'id': '7', 'name': 'Агроном'}]",False,"{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",None,False,None,2000,NaN,USD,True,None,None
2,106435824,False,Ландшафтный дизайнер/Садовник/Дендролог/Агроном,None,False,False,Москва,Открытая,None,None,2024-09-02T10:32:17+0300,2024-09-02T10:32:17+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,NaN,None,https://api.hh.ru/vacancies/106435824?host=hh.ru,https://hh.ru/vacancy/106435824,[],"{'id': '56479', 'name': 'Дэстэн Партнерс', 'ur...",{'requirement': 'Опыт работы в смежной должнос...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False,"[{'id': '34', 'name': 'Дизайнер, художник'}]",False,"{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",None,False,None,180000,NaN,RUR,False,None,None
3,106918725,False,Агроном,None,False,False,Москва,Открытая,None,None,2024-09-06T16:36:22+0300,2024-09-06T16:36:22+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,NaN,None,https://api.hh.ru/vacancies/106918725?host=hh.ru,https://hh.ru/vacancy/106918725,[],"{'id': '11412752', 'name': 'Здоровый Сад', 'ur...",{'requirement': 'Ответственность. Кропотливост...,None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],True,"[{'id': '7', 'name': 'Агроном'}]",False,"{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'part', 'name': 'Частичная занятость'}",None,False,None,30000,40000.0,RUR,False,Москва,улица Плеханова
4,105672373,False,Агроном/овощевод,None,False,False,Москва,Открытая,None,None,2024-08-28T12:27:02+0300,2024-08-28T12:27:02+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,NaN,None,https://api.hh.ru/vacancies/105672373?host=hh.ru,https://hh.ru/vacancy/105672373,[],"{'id': '2161246', 'name': 'МОСМАКС ТРЕТЬЯКОВСК...","{'requirement': 'Честность, добросовестность, ...",None,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False,"[{'id': '7', 'name': 'Агроном'}]",True,"{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",None,False,None,120000,NaN,RUR,False,None,None
5,107065128,False,Специалист по уходу за офисными растениями (са...,None,False,False,Москва,Открытая,None,None,2024-09-10T15:15:41+0300,2024-09-10T15:15:41+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,NaN,None,https://api.hh.ru/vacancies/107065128?host

Если Ваш запрос прошел успешно, то вы получите следующие ключи (значения данных ключей вы найдете по ссылке на документацию):

In [ ]:
data.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

Выберете для нас 5 наиболее интересных специальностей и получите по ним имеющиеся вакансии:

Посчитайте сколько всего найдено вакансий:

Создайте структуру данных, которая будет хранить в себе имя компании и ее координаты (широта и долгота):

Создайте экземпляр этой структуры данных для каждой компании, которая выставила вакансию:

Найдите количество вакансий, у которых заданы координаты (значения данных может быть пустым):

Отберите вакансии, которые расположены в пределах МКАД (можно грубо):

Постройте график в координатах "широта-долгота" для отображения вакансий внутри МКАД (не забывайте прописать прозрачность, чтобы увидеть скопления):

**Сделайте выводы о проделанной работе:**

Вам предстоит проанализировать датасет, содержащий информацию об бриллианта, их стоимости и остальных характеристиках.



Поясним значения хранящиеся в колонках:
*   `cut` - огранка бриллианта (относится к одной из примерно 10 наиболее распространенных)
*   `color` - цвет бриллианта, прозрачные бриллианты имеют градацию `D-Z`, "более высокие" буквы - более желтоватые, но часто имеют лучшие значения, поскольку цвет трудно определить один раз в кольце
*   `clarity` - чистота бриллианта, наличие включений (внутренних дефектов), чем меньше и мельче, тем лучше
*   `carat_weight` - масса бриллианта (слабо связана с размером бриллианта, т.к. огранка и качество огранки, как правило, играют большую роль)
*   `cut_quality` - качество ограники по системе оценки GIA Cut
*   `lab` - лаборатория оценивания
*   `meas_length` - длина бриллианта
*   `meas_width` - ширина бриллианта
*   `meas_depth` - глубина бриллианта
*   `total_sales_price` - цена бриллианта в долларах.
*    **и прочие колонки**

Скачаем данные:

In [ ]:
!gdown 10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8
# если работайте в jupyter то скачайте датасет с https://drive.google.com/uc?id=10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8

Создадим DataFrame:

In [ ]:
import pandas as pd

diamonds = pd.read_csv("diamonds.csv")

Выведете первые 5 строк датасета:

Найдите количество бриллиантов всего в данном датасете:

Выведите максимальную и минимальную цену бриллианта в формате `номер - цена`:

Посчитайте количество пропусков (пропуском считается значение `unknown` или `None`):

Найдите в каких столбцах присутствуют пропуски:

В нашем датасете присутствуют колонки, которые начинаются с `fancy_color_` или `fluor_`. Большинство значений в этих колонках - `unknown` или `None`, поэтому мы считаем их малоинформативными.

Ваша задача:

найти количество таких колонок, которые начинаются с `fancy_color_` или `fluor_`:

найти количество информативных (не `None` или `unknown`) значений в этих колонках:

удалить данные колонки из датасета, сохранив изменения (в дальнейшем работаем измененным датасетом):

Проделайте тоже самое для колонок, начинающихся с `culet_`:

Посчитайте сколько бы стоил (в тысячах долларов) сейф, в котором хранились бы все бриллианты из датасета:

Немного необоснованных манипуляций.

Выберите из датасета строки с четными индексами и выведете значения (не меняя нумерацию индексов):

А теперь с индексами, кратными 6:

Выберете из датасета только категорильные признаки и выведете датасет:

### Построение графиков

**Требования к графику:**

1.   График должен быть подписан
2.   Оси должны быть подписаны
3.   Должна быть легенда графика
4.   График должен быть информативным (желательно без сильных выбросов)

**Все надписи на русском языке!**

>**Примечание**
>
>График необходимо строить для очищенных данных - в них не должно быть пропусков по осям, по которым строите график. При этом **не перезатирайте** имеющийся датафрейм!

Постройте график `цвет бриллианта - стоимость`:

Постройте график `лабараторная оценка - кол-во бриллиантов`:

Постройте график `вес бриллианта - стоимость`:

Постройте график `система оценки GIA Cut - кол-во бриллиантов`:

Сделайте выводы по построенным графикам